In [43]:
#定义下游模型
import torch 
from transformers import AutoModel
from sentence_transformers import SentenceTransformer
pretrained = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
print("parameters: ",sum(i.numel() for i in pretrained.parameters()))
example = "随便说点啥，我要用来做模型试算嗯嗯"
feaures = pretrained.tokenize(example)
for key in feaures.keys():
    print(key, feaures[key].shape)
print(pretrained.forward(feaures)['sentence_embedding'].shape)
print(type(pretrained))
out1 = pretrained[0].forward(feaures)
out2 = pretrained[1].forward(feaures)
out3 = pretrained[2].forward(feaures)
print(out1.keys(),'\n',out2.keys(),'\n',out3.keys())
print(pretrained[0])

parameters:  22713216
input_ids torch.Size([17, 3])
token_type_ids torch.Size([17, 3])
attention_mask torch.Size([17, 3])
torch.Size([17, 384])
<class 'sentence_transformers.SentenceTransformer.SentenceTransformer'>
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'token_embeddings', 'sentence_embedding']) 
 dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'token_embeddings', 'sentence_embedding']) 
 dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'token_embeddings', 'sentence_embedding'])
Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 


In [ ]:
pretrained.encode(example).shape

In [47]:
from transformers import AutoTokenizer

#加载分词器
tokenizer = AutoTokenizer.from_pretrained('hfl/rbt6')
from transformers import AutoModel

#加载预训练模型
pretrained = AutoModel.from_pretrained('hfl/rbt6')
print(tokenizer)

#分词测试
inputs = tokenizer.batch_encode_plus(
    [[
        '海', '钓', '比', '赛', '地', '点', '在', '厦', '门', '与', '金', '门', '之', '间',
        '的', '海', '域', '。'
    ],
     [
         '这', '座', '依', '山', '傍', '水', '的', '博', '物', '馆', '由', '国', '内', '一',
         '流', '的', '设', '计', '师', '主', '持', '设', '计', '，', '整', '个', '建', '筑',
         '群', '精', '美', '而', '恢', '宏', '。'
     ]],
    truncation=True,
    padding=True,
    return_tensors='pt',
    is_split_into_words=True)

print(inputs.keys())
for i in inputs:
    print(i,inputs[i].shape)
pretrained(**inputs).last_hidden_state.shape
#del out3['token_embeddings'],out3['sentence_embedding']
pretrained(**inputs).last_hidden_state.shape

Some weights of the model checkpoint at hfl/rbt6 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no tru

BertTokenizerFast(name_or_path='hfl/rbt6', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
input_ids torch.Size([2, 37])
token_type_ids torch.Size([2, 37])
attention_mask torch.Size([2, 37])


torch.Size([2, 37, 768])

In [63]:

#模型试算
#[b, lens] -> [b, lens, 768]
#pretrained(**inputs).last_hidden_state.shape
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.tuneing = False
        self.pretrained = bertmodel

        self.rnn = torch.nn.GRU(768, 768,batch_first=True)
        self.fc = torch.nn.Linear(768, 8)

    def forward(self, inputs):
        #print(inputs.keys())
        for i in inputs:
            print(inputs[i].size())
        if self.tuneing:
            out = self.pretrained(**inputs).last_hidden_state
        else:
            with torch.no_grad():
                out = pretrained(**inputs).last_hidden_state

        out, _ = self.rnn(out)

        out = self.fc(out).softmax(dim=2)

        return out

    def fine_tuneing(self, tuneing):
        self.tuneing = tuneing
        if tuneing:
            for i in pretrained.parameters():
                i.requires_grad = True

            pretrained.train()
            self.pretrained = pretrained
        else:
            for i in pretrained.parameters():
                i.requires_grad_(False)

            pretrained.eval()
            self.pretrained = None


model = Model()


In [27]:
# test pooling 
import torch
a = torch.ones(100).reshape((1,4,25))
print(a.shape)
b  = torch.nn.AvgPool1d(kernel_size=1,stride=10)
c  = torch.nn.Conv2d(in_channels = 1, out_channels = 2, kernel_size=(2,1),stride=(2,1))
c(a).shape

torch.Size([1, 4, 25])


torch.Size([2, 2, 25])

In [70]:
import json 
path = './cail/competition_stage_2_train.json'
file = open(path,'r',encoding='utf-8')
line = file.readline()
cases = json.loads(line)


In [95]:
cases
print(type(cases['Case_A']))
print(type(cases['Case_A'][0]))
print("".join(cases['Case_A']))

<class 'list'>
<class 'str'>
上海市长宁区人民检察院指控:2020年3月,被告人XXX受他人指使在广东省揭阳市使用自己的身份证办理中国农业银行卡、手机卡及网银U盾1套,并以人民币1,000元的价格出售给他人。该中国农业银行卡之后被用于诈骗活动。2020年4月,被害人XXX在本市长宁区武夷路家中被电信网络诈骗,钱款经一级账户XXX的平安银行卡转账至被告人方俊河办理的农业银行卡,共计人民币120万元。该农业银行卡支付结算金额超人民币1,860余万元。2020年7月23日,被告人方俊河在广东省揭阳市榕城区榕东旧寨村XXX某某微电机厂内被民警抓获。被告人XXX到案后如实供述上述犯罪事实。  上述事实,被告人方俊河在开庭审理过程中亦无异议,并有被害人柏某某的陈述、平安银行出具的客户个人信息、个人交易明细,中国农业银行出具的个人开卡业务申请书、资料修改业务申请书,平安银行出具的个人交易明细及中国农业银行出具的银行卡交易明细清单,上海市***长宁分局扣押清单、扣押笔录及扣押照片,上海市***长宁分局案发经过表格、抓获经过、个人信息及十指指纹信息卡等证据证实,足以认定。上述事实,被告人方俊河在开庭审理过程中亦无异议,并有被害人柏某某的陈述、平安银行出具的客户个人信息、个人交易明细,中国农业银行出具的个人开卡业务申请书、资料修改业务申请书,平安银行出具的个人交易明细及中国农业银行出具的银行卡交易明细清单,上海市***长宁分局扣押清单、扣押笔录及扣押照片,上海市***长宁分局案发经过表格、抓获经过、个人信息及十指指纹信息卡等证据证实,足以认定。本院认为,被告人XXX明知他人利用信息网络实施犯罪,为其犯罪提供支付结算帮助,情节严重,其行为已构成帮助信息网络犯罪活动罪,依法应予惩处。公诉机关的指控,事实清楚,定性正确。被告人XXX到案后如实供述自己的罪行,依法予以从轻处罚。辩护人与此相关的辩护意见,本院予以采纳


In [112]:
file = open(path,'r',encoding='utf-8')
cases = file.readlines()
cases = [json.loads(i) for i in cases]
cases[0]['Case_A']

['上海市长宁区人民检察院指控:2020年3月,被告人XXX受他人指使在广东省揭阳市使用自己的身份证办理中国农业银行卡、手机卡及网银U盾1套,并以人民币1,000元的价格出售给他人。',
 '该中国农业银行卡之后被用于诈骗活动。',
 '2020年4月,被害人XXX在本市长宁区武夷路家中被电信网络诈骗,钱款经一级账户XXX的平安银行卡转账至被告人方俊河办理的农业银行卡,共计人民币120万元。',
 '该农业银行卡支付结算金额超人民币1,860余万元。',
 '2020年7月23日,被告人方俊河在广东省揭阳市榕城区榕东旧寨村XXX某某微电机厂内被民警抓获。',
 '被告人XXX到案后如实供述上述犯罪事实。',
 '  上述事实,被告人方俊河在开庭审理过程中亦无异议,并有被害人柏某某的陈述、平安银行出具的客户个人信息、个人交易明细,中国农业银行出具的个人开卡业务申请书、资料修改业务申请书,平安银行出具的个人交易明细及中国农业银行出具的银行卡交易明细清单,上海市***长宁分局扣押清单、扣押笔录及扣押照片,上海市***长宁分局案发经过表格、抓获经过、个人信息及十指指纹信息卡等证据证实,足以认定。',
 '上述事实,被告人方俊河在开庭审理过程中亦无异议,并有被害人柏某某的陈述、平安银行出具的客户个人信息、个人交易明细,中国农业银行出具的个人开卡业务申请书、资料修改业务申请书,平安银行出具的个人交易明细及中国农业银行出具的银行卡交易明细清单,上海市***长宁分局扣押清单、扣押笔录及扣押照片,上海市***长宁分局案发经过表格、抓获经过、个人信息及十指指纹信息卡等证据证实,足以认定。',
 '本院认为,被告人XXX明知他人利用信息网络实施犯罪,为其犯罪提供支付结算帮助,情节严重,其行为已构成帮助信息网络犯罪活动罪,依法应予惩处。',
 '公诉机关的指控,事实清楚,定性正确。',
 '被告人XXX到案后如实供述自己的罪行,依法予以从轻处罚。',
 '辩护人与此相关的辩护意见,本院予以采纳']

In [131]:
from torch.utils.data import Dataset,DataLoader
class CailDataset(Dataset):
    def __init__(self,dataset) -> None:
        super(CailDataset).__init__()
        self.dataset = dataset
        file = open(dataset,'r',encoding='utf-8')
        cases = file.readlines()
        self.cases = [json.loads(i) for i in cases]
        self.length = len(self.cases)
    def __getitem__(self, index) :
        # caseA = self.cases[index]["Case_A"]
        # caseB = self.cases[index]["Case_B"]
        # label = self.cases[index]["label"]
        # return caseA,caseB,label
        return index 
    def __len__(self):
        return self.length

In [126]:

#模型试算
#[b, lens] -> [b, lens, 768]
#pretrained(**inputs).last_hidden_state.shape
from sentence_transformers import SentenceTransformer
pretrained = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
class CailModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.tuneing = False
        self.pretrained = pretrained
        self.fc = torch.nn.Linear(384,3)

    def forward(self, inputs):
        #print(inputs.keys())
        # for i in inputs:
        #     print(inputs[i].size())
        #print(type(inputs))
        emba = pretrained.encode("".join(inputs['Case_A']))
        embb = pretrained.encode("".join(inputs['Case_B']))
        out = emba+embb
        out = torch.FloatTensor(out)
        out = self.fc(out)
        #out = torch.argmax(out).long()
        return out

    def fine_tuneing(self, tuneing):
        self.tuneing = tuneing
        if tuneing:
            for i in pretrained.parameters():
                i.requires_grad = True

            pretrained.train()
            self.pretrained = pretrained
        else:
            for i in pretrained.parameters():
                i.requires_grad_(False)

            pretrained.eval()
            self.pretrained = None
model = CailModel()

In [133]:
file = open(path,'r',encoding='utf-8')
cases = file.readlines()
cases = [json.loads(i) for i in cases]
file.close()
epoch  = 100 
from transformers import AdamW
lr = 2e-5
optimizer = AdamW(model.parameters(), lr=lr)
from torch import nn 
length = len(cases)
from tqdm import trange 
CE_loss = nn.CrossEntropyLoss()     #定义损失函数
for i in range(epoch):
    losses  = 0 
    correct = 0 
    for index  in trange(length):
        output  = model(cases[index])
        label   = torch.tensor(cases[index]['label'])
        loss = CE_loss(output,label)
        losses+=loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if torch.argmax(output)==label:
            correct+=1
    print('epoch '+str(i)+'loss',losses/length)
    print(correct/length)
    if i%10==0 or i == epoch-1:
        torch.save(model,str(i)+"th_model.model")

d:\programs\anaconda\envs\pytorch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 3000/3000 [01:03<00:00, 47.53it/s]


epoch 0loss tensor(1.0360, grad_fn=<DivBackward0>)
0.46066666666666667


100%|██████████| 3000/3000 [00:57<00:00, 51.83it/s]


epoch 1loss tensor(1.0357, grad_fn=<DivBackward0>)
0.4623333333333333


100%|██████████| 3000/3000 [00:54<00:00, 55.54it/s]


...

epoch 97loss tensor(1.0115, grad_fn=<DivBackward0>)
0.49666666666666665


100%|██████████| 3000/3000 [00:55<00:00, 54.05it/s]


epoch 98loss tensor(1.0114, grad_fn=<DivBackward0>)
0.49633333333333335


100%|██████████| 3000/3000 [00:55<00:00, 54.29it/s]


epoch 99loss tensor(1.0112, grad_fn=<DivBackward0>)
0.49633333333333335


In [130]:
torch.save(model,"binaryclass.model")

In [132]:
trainset = CailDataset(dataset="./cail/competition_stage_2_train.json")
train_loader = DataLoader(trainset,batch_size=16,shuffle=True)
from tqdm import trange
correct = 0 
file = open(path,'r',encoding='utf-8')
cases = file.readlines()
cases = [json.loads(i) for i in cases]
file.close()
for  i,data in enumerate(train_loader):
    # print(type(data))
    # print(data.keys())
    # print(data['Case_A'])
    # label = torch.LongTensor(data['label'])
    # fake = model(data)
    # correct+=sum(fake,label
# print(correct)


tensor([2634,  262, 2724, 1535,  279, 1246, 1000, 1783,  273,  821,  786,  509,
        1348, 1634, 1054, 1952])
tensor([2447, 2556,  408, 2412,   52, 2943,   80, 2304, 2911, 1719,  423, 2639,
        2659, 1633,  340, 1957])
tensor([2503, 1338, 2368, 2230, 1523, 1746, 1395, 2184, 1890, 2180, 2762, 2768,
        2359, 2494,  413, 1259])
tensor([ 380,  412, 2799, 1350,  901, 1230, 1697, 1180,  252,  109,  353, 2029,
        1047,  798, 2156, 2459])
tensor([1787, 1074, 2490, 1818,  456,  505, 1362,  870, 1138,  940, 1391, 2849,
        1087, 2636, 2314, 2638])
tensor([ 777, 1946,  961,   33,  991, 1142,  619, 1711, 1514,  199, 2982,  278,
         467,  702, 1516,  770])
tensor([1083, 2301, 1521, 1683, 1186,  259,  888, 1569, 2101, 1465,  116, 2641,
         585, 1994, 1211, 1490])
tensor([1561, 1657, 1092, 2660,   15, 2248, 2872,  620, 1382, 2052, 2998,  889,
        2193, 1183, 1953, 1217])
tensor([2824, 2008, 1254,  748,  101, 1158, 1547,   28,  125, 2767, 2528, 1802,
         736, 29